<a href="https://colab.research.google.com/github/nasibehmohammadi/Thesis/blob/main/1GRU_d3(UCI).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# 1️⃣ بارگذاری دیتاست شخصی (لطفاً نام فایل را تغییر بده!)
file_path = "/content/d3.csv"  # نام فایل خود را اینجا بگذار
df = pd.read_csv(file_path)

# 2️⃣ بررسی ساختار داده‌ها
print("🔍 بررسی ۵ سطر اول دیتاست:")
print(df.head())

# 3️⃣ مشخص کردن ویژگی‌های ورودی و خروجی
# نام ستون‌ها را بر اساس دیتاست خود تغییر بده!
feature_columns = ["B","C"]  # نام ویژگی‌ها
target_column = "A"  # نام ستون هدف (برچسب‌ها)

# تبدیل دیتافریم به آرایه NumPy
X_data = df[feature_columns].values
y_data = df[target_column].values

# مشخص کردن تعداد ویژگی‌ها
num_features = X_data.shape[1]

# 4️⃣ تنظیم داده‌ها برای مدل GRU
time_steps = 10  # تعداد مراحل زمانی (مثلاً ۱۰ ورود اخیر هر کاربر)

# ایجاد دنباله‌های زمانی برای GRU
X_sequences = []
y_sequences = []

for i in range(len(X_data) - time_steps):
    X_sequences.append(X_data[i : i + time_steps])  # داده‌های ۱۰ زمان اخیر
    y_sequences.append(y_data[i + time_steps])  # مقدار هدف بعد از ۱۰ مرحله

X_sequences = np.array(X_sequences)
y_sequences = np.array(y_sequences)

# 5️⃣ تقسیم داده‌ها به مجموعه آموزش و تست
X_train, X_test, y_train, y_test = train_test_split(X_sequences, y_sequences, test_size=0.2, random_state=42)

# 6️⃣ مقیاس‌بندی داده‌ها
scaler = MinMaxScaler()
X_train = X_train.reshape(-1, num_features)
X_test = X_test.reshape(-1, num_features)

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

X_train = X_train.reshape(-1, time_steps, num_features)
X_test = X_test.reshape(-1, time_steps, num_features)

# 7️⃣ ایجاد مدل GRU
model = Sequential([
    GRU(64, return_sequences=True, input_shape=(time_steps, num_features)),
    Dropout(0.2),
    GRU(32, return_sequences=False),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1, activation='sigmoid')
])

# 8️⃣ کامپایل مدل
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 9️⃣ آموزش مدل روی دیتای شخصی
history = model.fit(X_train, y_train, epochs=2, batch_size=32, validation_data=(X_test, y_test))

# 🔟 ارزیابی مدل
loss, accuracy = model.evaluate(X_test, y_test)
print(f"✅ دقت مدل روی داده‌های تست: {accuracy * 100:.2f}%")

# ۱۱️⃣ پیش‌بینی روی یک نمونه جدید
test_sample = X_test[0].reshape(1, time_steps, num_features)
prediction = model.predict(test_sample)

if prediction > 0.5:
    print("🚨 ورود مشکوک! نیاز به احراز هویت بیشتر.")
else:
    print("✅ ورود ایمن است.")

🔍 بررسی ۵ سطر اول دیتاست:
   A      B      C
0  0  11044  45813
1  1  11044  65766
2  1   9521  72531
3  1   9521  72531
4  1   9521  72531


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/2
17902/17902 ━━━━━━━━━━━━━━━━━━━━ 304s 17ms/step - accuracy: 0.9837 - loss: 0.0846 - val_accuracy: 0.9851 - val_loss: 0.0774
Epoch 2/2
17902/17902 ━━━━━━━━━━━━━━━━━━━━ 323s 17ms/step - accuracy: 0.9847 - loss: 0.0799 - val_accuracy: 0.9851 - val_loss: 0.0774
4476/4476 ━━━━━━━━━━━━━━━━━━━━ 21s 5ms/step - accuracy: 0.9851 - loss: 0.0775
✅ دقت مدل روی داده‌های تست: 98.51%
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 397ms/step
🚨 ورود مشکوک! نیاز به احراز هویت بیشتر.


In [3]:
from sklearn.metrics import f1_score

In [4]:
from sklearn.metrics import f1_score

# 1️⃣ پیش‌بینی بر روی مجموعه تست
y_pred = model.predict(X_test)

# 2️⃣ تبدیل احتمالات به برچسب‌های 0 و 1 (اگر مقدار احتمال > 0.5 باشد، کلاس 1 است، در غیر این صورت 0)
y_pred_classes = (y_pred > 0.5).astype(int)

# 3️⃣ محاسبه F1-Score
f1 = f1_score(y_test, y_pred_classes)

print(f"🎯 F1-Score مدل: {f1:.4f}")


4476/4476 ━━━━━━━━━━━━━━━━━━━━ 17s 4ms/step
🎯 F1-Score مدل: 0.9925
